**Imports**

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import rasterio as rio
import geopandas as gpd
from rasterio.plot import show
from rasterio.transform import from_bounds

In [6]:
class Box:
    def __init__(self, bounds=None, left:int=0, bottom:int=0, right:int=0, top:int=0):
        if bounds is not None: self.left,self.bottom,self.right,self.top = bounds
        else: self.left,self.bottom,self.right,self.top = left,bottom,right,top
        self.width, self.height = self.right -self.left, self.top -self.bottom
    def __str__(self):
        return "left:{} bottom:{} right:{} top:{} width:{} height:{}".format(
            self.left, self.bottom, self.right, self.top, self.width, self.height)
    def in_bounds(self, x:int, y:int) -> bool:
        return self.left < x < self.right and self.bottom < y < self.top

**Cropper**

In [2]:
def cropper(origin, new_name, x, y, new_width, new_height):
    origin_arr = np.array(origin.read(1))
    origin_bounds = origin.bounds
    
    left = int(x * width)
    right = int((x+1) * width)
    bottom = int((y+1) * height)
    top = int(y * height)
        
    new_left = origin_bounds.left + left
    new_right = origin_bounds.left + right
    new_bottom = origin_bounds.top - bottom
    new_top = origin_bounds.top - top
    
    print(f"[{x},{y}]: {new_left}{new_right}|{new_bottom}{new_top}")
    # crop out of the array
    new_arr = origin_arr[left:right,top:bottom]
    
    new_meta = origin.meta
    
    # transform needs to be edited so the crop has correct coords
    new_meta['transform'](
        # new left-most coord
        new_left, 
        # new bottom-most coord
        new_bottom, 
        # new right-most coord
        new_right, 
        # new top-most coord
        new_top, 
        # new width
        new_width,
        # new height
        new_height)
    
    new_meta["width"] = new_width
    new_meta["height"] = new_height
    
    new_path = os.path.join("/media/seppe/DiskSpaceInvader/3DHouseData",new_name)
    # open a new file in write mode as destination
    # **meta is the **kwargs passed to the tif
    with rio.open(new_path, "w", **new_meta) as destination:
        # write the pixel data with the **meta data
        destination.write(new_arr, indexes=1)
        print("new file: ",new_name)
        destination.close()
    return

**Splitter**

In [3]:
def splitter(tiff_path:str, div:int = 10):
    # read the name of the file, without the file extension, and save it as name
    name = os.path.split(tiff_path)[1][:-4]
    
    tiff = rio.open(tiff_path)
    tiff_arr = np.array(tiff.read(1))
    tiff_meta = tiff.meta
    origin_bounds = tiff.bounds
    
    origin_width = origin_bounds.right - origin_bounds.left
    origin_height = origin_bounds.top - origin_bounds.bottom
    print(origin_width, origin_height)
    
    if (origin_width%div == 0) and (origin_height%div == 0):
        width_div = origin_width/div
        height_div = origin_height/div
        for i in range(div):
            for j in range(div):                
                # call the cropper function, with a new filename, and the current i and j
                cropper(tiff, 
                        f"{name}cutx{i}y{j}.tif", 
                        i, j, 
                        width_div, height_div)
        tiff.close()
        os.remove(tiff_path)
    else:
        print("error in: ", tiff_path, " These bounds can't be divided by ", str(div))
    return

    


*Maarten's Subdivide*

In [ ]:
k15_path = "/home/becode/Projects/3D_House/LIDAR/k15/DHMVIIDSMRAS1m_k15/GeoTIFF/DHMVIIDSMRAS1m_k15.tif"
k15_tif = rio.open(k15_path)
subdivide("./k15", k15_tif, 5)
print("Done!")

**File Finder**

In [9]:
# get the relative path of the current project that this notebook is in
project_path = os.path.abspath(os.path.join("readfiles.ipynb", os.pardir, os.pardir))
int_data_path = os.path.join(project_path, "Data")

# Insert your own data path here, this is my external hard drive
ext_data_path = "/media/seppe/DiskSpaceInvader/3DHouseData"

sub_lookup = {} # !!! Should write to dataframe directly

def subdivide(name:str, tif:rio.io.DatasetReader, sub:int=2) -> None:
    if sub is 0: return
    
    arr = np.array(tif.read(1))
    meta = tif.meta
    bounds = tif.bounds
    
    meta["width"]  /= 2
    meta["height"] /= 2
    
    sub_bound = {
        0:(0,1,-1,0),
        1:(1,1,0,0),
        2:(0,0,-1,-1),
        3:(1,0,0,-1)}
    
    sub_slice = {
        0:{"sx":slice(None,int(meta["height"])),
           "sy":slice(None,int(meta["width"]))},
        1:{"sx":slice(None,int(meta["height"])),
           "sy":slice(int(meta["width"]),None)},
        2:{"sx":slice(int(meta["height"]),None),
           "sy":slice(None,int(meta["width"]))},
        3:{"sx":slice(int(meta["height"]),None),
           "sy":slice(int(meta["width"]),None)}}
    
    for idx in range(4):        
        meta["transform"] = from_bounds(
            bounds.left + meta["width"] * sub_bound[idx][0],
            bounds.bottom + meta["height"] * sub_bound[idx][1],
            bounds.right + meta["width"] * sub_bound[idx][2],
            bounds.top + meta["height"] * sub_bound[idx][3],
            meta["width"], meta["height"]
        )
        with rio.open(name +f"_{idx}.tif", "w+", **meta) as subdiv:
            subdiv.write(arr[sub_slice[idx]["sx"],sub_slice[idx]["sy"]], indexes=1)
            subdivide(name +f"_{idx}", subdiv, sub-1)
            if sub > 1: os.remove(name +f"_{idx}.tif")
            else: sub_lookup[name[2:] +f"_{idx}"] = Box(subdiv.bounds)
    return

def count_files(fpath:str, filetype:str):
    print(f"looking for {filetype} files in: " + fpath)
    # Keep track of the number of files found
    count = 0
    # Walk through the folder containing the data, and check if the files match either .tif or .shp
    # If so; open that file, and print how many there were found
    for root, dirs, files in os.walk(fpath):
        for name in files:
            if name.endswith(filetype):
                # add to the tif counter
                count += 1
    return count

total_tifs = count_files(ext_data_path, ".tif")
print(f"{total_tifs} .tif files")

def find_file(fpath:str):
    print("looking for files in: " + fpath)
    # Keep track of the number of files found
    tifs = 0
    shps = 0
    # Walk through the folder containing the data, and check if the files match either .tif or .shp
    # If so; open that file, and print how many there were found
    for root, dirs, files in os.walk(fpath):
        for name in files:
            if name.endswith(".tif"):
                # add to the tif counter
                tifs += 1
                # read the name of the file, without the file extension, and save it as name
                file_name = os.path.join(root, name)[:-4]
                print(f"Dividing file {tifs} out of {total_tifs}: {file_name}")
                tif = rio.open(os.path.join(root, name))
                subdivide(file_name, tif, 5)
                tif.close()
                print("ok")
            if name.endswith(".shp"):
                # Optional print, used for debugging purposes
                # print(os.path.join(root, name))
                shps += 1
    # Print the number of files found
    print("Found ", tifs, " .tif files.")
    print("Found ", shps, " .shps files.")
    return sub_lookup

#find_file(ext_data_path)


looking for .tif files in: /media/seppe/DiskSpaceInvader/3DHouseData
88150 .tif files
